In [3]:
import geopandas as gpd

In [4]:
data = gpd.read_file('nodos.geojson')
data

,ADDR_TYPE,link_id,L_ADDRFORM,L_ADDRSCH,L_NREFADDR,L_REFADDR,R_ADDRFORM,R_ADDRSCH,R_NREFADDR,R_REFADDR,ST_LANGCD,ST_NAME,ST_NM_BASE,ST_NM_SUFF,ST_TYP_AFT,ST_TYP_ATT,ST_TYP_BEF,geometry
0,B,1115212395,None,None,None,None,N,E,672,30,SPA,CALLE MARIANO ZÚÑIGA,MARIANO ZÚÑIGA,None,None,N,CALLE,"LINESTRING (-99.63067 19.26921, -99.63059 19.2..."
1,B,702663287,N,E,100,132,N,O,101,125,SPA,CALLE CARLOTA,CARLOTA,None,None,N,CALLE,"LINESTRING (-99.63421 19.26966, -99.63322 19.2..."
2,None,1272517807,None,None,None,None,None,None,None,None,None,None,None,None,None,N,None,"LINESTRING (-99.6323 19.26838, -99.63215 19.26..."
3,None,1115212390,None,None,None,None,None,None,None,None,None,None,None,None,None,N,None,"LINESTRING (-99.63069 19.26954, -99.63077 19.2..."
4,B,702699915,N,E,128,148,N,O,121,139,SPA,CALLE ANA MARÍA,ANA MARÍA,None,None,N,CALLE,"LINESTRING (-99.63456 19.26988, -99.63379 19.2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760184,None,977718069,None,None,None,None,None,None,None,None,SPA,PASEO DEL BOSQUE,BOSQUE,None,None,N,PASEO DEL,"LINESTRING (-98.96286 19.72809, -98.9631 19.72..."
760185,None,1116108951,None,None,None,None,None,None,None,None,SPA,FRESNOS,FRESNOS,None,None,N,None,"LINESTRING (-98.96102 19.72808, -98.96096 19.7..."
760186,None,834254981,None,None,None,None,None,None,None,None,None,None,None,None,None,N,None,"LINESTRING (-98.96171 19.72798, -98.9614 19.72..."
760187,None,1116108356,None,None,None,None,None,None,None,None,SPA,FRESNOS,FRESNOS,None,None,N,None,"LINESTRING (-98.96108 19.72817, -98.96102 19.7..."


In [14]:
import geopandas as gpd
import pandas as pd
import os
import glob
from pathlib import Path
import logging

def setup_logging():
    """Setup logging configuration"""
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )
    return logging.getLogger(__name__)

def find_geojson_files(directories):
    """
    Find all .geojson files in the specified directories
    
    Args:
        directories (list): List of directory paths to search
    
    Returns:
        list: List of full paths to .geojson files
    """
    geojson_files = []
    
    for directory in directories:
        if os.path.exists(directory):
            # Use glob to find all .geojson files recursively
            pattern = os.path.join(directory, "**", "*.geojson")
            files = glob.glob(pattern, recursive=True)
            geojson_files.extend(files)
            logging.info(f"Found {len(files)} .geojson files in {directory}")
        else:
            logging.warning(f"Directory not found: {directory}")
    
    return geojson_files

def read_geojson_safe(file_path):
    """
    Safely read a GeoJSON file with error handling
    
    Args:
        file_path (str): Path to the GeoJSON file
    
    Returns:
        geopandas.GeoDataFrame or None: The loaded GeoDataFrame or None if failed
    """
    try:
        gdf = gpd.read_file(file_path)
        logging.info(f"Successfully loaded: {os.path.basename(file_path)} ({len(gdf)} features)")
        return gdf
    except Exception as e:
        logging.error(f"Failed to load {file_path}: {str(e)}")
        return None

def add_source_column(gdf, file_path):
    """
    Add a source column to track which file each feature came from
    
    Args:
        gdf (geopandas.GeoDataFrame): The GeoDataFrame
        file_path (str): Path to the source file
    
    Returns:
        geopandas.GeoDataFrame: GeoDataFrame with source column added
    """
    if gdf is not None:
        # Add source file information
        gdf['source_file'] = os.path.basename(file_path)
        gdf['source_folder'] = os.path.basename(os.path.dirname(file_path))
    return gdf

def merge_geodataframes(gdfs, how='outer'):
    """
    Merge multiple GeoDataFrames into one
    
    Args:
        gdfs (list): List of GeoDataFrames to merge
        how (str): How to handle column differences ('outer', 'inner')
    
    Returns:
        geopandas.GeoDataFrame: Merged GeoDataFrame
    """
    if not gdfs:
        logging.warning("No valid GeoDataFrames to merge")
        return None
    
    if len(gdfs) == 1:
        return gdfs[0]
    
    try:
        # Concatenate all GeoDataFrames
        merged_gdf = pd.concat(gdfs, ignore_index=True, sort=False)
        
        # Ensure it's still a GeoDataFrame
        if not isinstance(merged_gdf, gpd.GeoDataFrame):
            merged_gdf = gpd.GeoDataFrame(merged_gdf)
        
        logging.info(f"Successfully merged {len(gdfs)} files into {len(merged_gdf)} total features")
        return merged_gdf
    
    except Exception as e:
        logging.error(f"Failed to merge GeoDataFrames: {str(e)}")
        return None

def save_merged_geojson(gdf, output_path):
    """
    Save the merged GeoDataFrame as a GeoJSON file
    
    Args:
        gdf (geopandas.GeoDataFrame): The GeoDataFrame to save
        output_path (str): Path where to save the file
    """
    try:
        # Ensure output directory exists (only if there's a directory in the path)
        output_dir = os.path.dirname(output_path)
        if output_dir:  # Only create directory if it's not empty
            os.makedirs(output_dir, exist_ok=True)
        
        # Save as GeoJSON
        gdf.to_file(output_path, driver='GeoJSON')
        logging.info(f"Merged GeoJSON saved to: {output_path}")
        
        # Print summary statistics
        print(f"\nMerge Summary:")
        print(f"- Total features: {len(gdf)}")
        print(f"- Total columns: {len(gdf.columns)}")
        print(f"- Coordinate Reference System: {gdf.crs}")
        print(f"- Output file: {os.path.abspath(output_path)}")
        
    except Exception as e:
        logging.error(f"Failed to save merged file: {str(e)}")
        # Try alternative approach - save in current directory with a simpler name
        try:
            alternative_path = "merged_geojson_output.geojson"
            gdf.to_file(alternative_path, driver='GeoJSON')
            logging.info(f"Successfully saved to alternative path: {alternative_path}")
            print(f"File saved as: {os.path.abspath(alternative_path)}")
        except Exception as e2:
            logging.error(f"Alternative save also failed: {str(e2)}")

def main():
    """Main function to execute the GeoJSON merging process"""
    
    # Setup logging
    logger = setup_logging()
    
    # Define input directories
    directories = [
        'Student Sample Data/STREETS_NAV'
    ]
    
    # Define output file path
    output_file = 'streets_nav.geojson'
    
    logger.info("Starting GeoJSON files merger...")
    
    # Find all GeoJSON files
    geojson_files = find_geojson_files(directories)
    
    if not geojson_files:
        logger.warning("No .geojson files found in the specified directories")
        return
    
    logger.info(f"Found {len(geojson_files)} .geojson files total")
    
    # Read all GeoJSON files
    geodataframes = []
    for file_path in geojson_files:
        gdf = read_geojson_safe(file_path)
        if gdf is not None:
            # Add source information
            gdf = add_source_column(gdf, file_path)
            geodataframes.append(gdf)
    
    # Merge all GeoDataFrames
    merged_gdf = merge_geodataframes(geodataframes)
    
    if merged_gdf is not None:
        # Save the merged result
        save_merged_geojson(merged_gdf, output_file)
        
        # Optional: Display column information
        print(f"\nColumns in merged dataset:")
        for col in merged_gdf.columns:
            print(f"- {col}")
            
        # Optional: Show data types
        print(f"\nData types:")
        print(merged_gdf.dtypes)
        
    else:
        logger.error("Failed to create merged GeoDataFrame")

# Additional utility functions for analysis
def analyze_merged_data(gdf):
    """
    Perform basic analysis on the merged GeoDataFrame
    
    Args:
        gdf (geopandas.GeoDataFrame): The merged GeoDataFrame
    """
    print(f"\n=== Data Analysis ===")
    print(f"Total features: {len(gdf)}")
    print(f"Coordinate Reference System: {gdf.crs}")
    print(f"Bounding box: {gdf.total_bounds}")
    
    # Count features by source
    if 'source_folder' in gdf.columns:
        source_counts = gdf['source_folder'].value_counts()
        print(f"\nFeatures by source folder:")
        for folder, count in source_counts.items():
            print(f"- {folder}: {count}")
    
    # Basic geometry statistics
    if hasattr(gdf, 'geometry'):
        geom_types = gdf.geometry.type.value_counts()
        print(f"\nGeometry types:")
        for geom_type, count in geom_types.items():
            print(f"- {geom_type}: {count}")

if __name__ == "__main__":
    main()

2025-05-17 21:42:15,035 - INFO - Starting GeoJSON files merger...
2025-05-17 21:42:15,036 - INFO - Found 20 .geojson files in Student Sample Data/STREETS_NAV
2025-05-17 21:42:15,037 - INFO - Found 20 .geojson files total
2025-05-17 21:42:15,511 - INFO - Successfully loaded: SREETS_NAV_4815440.geojson (11961 features)
2025-05-17 21:42:16,864 - INFO - Successfully loaded: SREETS_NAV_4815079.geojson (36140 features)
2025-05-17 21:42:17,460 - INFO - Successfully loaded: SREETS_NAV_4815084.geojson (13683 features)
2025-05-17 21:42:21,772 - INFO - Successfully loaded: SREETS_NAV_4815085.geojson (121060 features)
2025-05-17 21:42:21,891 - INFO - Successfully loaded: SREETS_NAV_4815441.geojson (2953 features)
2025-05-17 21:42:22,219 - INFO - Successfully loaded: SREETS_NAV_4815078.geojson (7782 features)
2025-05-17 21:42:26,478 - INFO - Successfully loaded: SREETS_NAV_4815087.geojson (115376 features)
2025-05-17 21:42:27,752 - INFO - Successfully loaded: SREETS_NAV_4815097.geojson (37644 featu


Merge Summary:
- Total features: 760189
- Total columns: 44
- Coordinate Reference System: EPSG:4326
- Output file: /Users/cris/Desktop/GUADALAHACKS/streets_nav.geojson

Columns in merged dataset:
- AR_AUTO
- AR_BUS
- AR_CARPOOL
- AR_DELIV
- AR_EMERVEH
- AR_MOTOR
- AR_PEDEST
- AR_TAXIS
- AR_TRAFF
- AR_TRUCKS
- BRIDGE
- CONTRACC
- COVERIND
- DIR_TRAVEL
- DIVIDER
- FERRY_TYPE
- FROM_LANES
- FRONTAGE
- FR_SPD_LIM
- FUNC_CLASS
- INDESCRIB
- INTERINTER
- LANE_CAT
- link_id
- LOW_MBLTY
- MANOEUVRE
- MULTIDIGIT
- PAVED
- POIACCESS
- PRIORITYRD
- PRIVATE
- PUB_ACCESS
- RAMP
- ROUNDABOUT
- SPEED_CAT
- TOLLWAY
- TO_LANES
- TO_SPD_LIM
- TUNNEL
- UNDEFTRAFF
- URBAN
- geometry
- source_file
- source_folder

Data types:
AR_AUTO            object
AR_BUS             object
AR_CARPOOL         object
AR_DELIV           object
AR_EMERVEH         object
AR_MOTOR           object
AR_PEDEST          object
AR_TAXIS           object
AR_TRAFF           object
AR_TRUCKS          object
BRIDGE             obje

In [11]:
sn_data = gpd.read_file('streets_naming_joint.geojson')
sn_data

,ADDR_TYPE,link_id,L_ADDRFORM,L_ADDRSCH,L_NREFADDR,L_REFADDR,R_ADDRFORM,R_ADDRSCH,R_NREFADDR,R_REFADDR,ST_LANGCD,ST_NAME,ST_NM_BASE,ST_NM_SUFF,ST_TYP_AFT,ST_TYP_ATT,ST_TYP_BEF,source_file,source_folder,geometry
0,None,1356100350,None,None,None,None,None,None,None,None,None,None,None,None,None,N,None,SREETS_NAMING_ADDRESSING_4815083.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.53179 19.51172, -99.53184 19.5..."
1,None,1356100347,None,None,None,None,None,None,None,None,None,None,None,None,None,N,None,SREETS_NAMING_ADDRESSING_4815083.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.53872 19.51172, -99.53862 19.5..."
2,None,1356100338,None,None,None,None,None,None,None,None,None,None,None,None,None,N,None,SREETS_NAMING_ADDRESSING_4815083.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.54132 19.51172, -99.54134 19.5..."
3,None,1356100349,None,None,None,None,None,None,None,None,None,None,None,None,None,N,None,SREETS_NAMING_ADDRESSING_4815083.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.53243 19.51222, -99.53252 19.5..."
4,None,1356100348,None,None,None,None,None,None,None,None,None,None,None,None,None,N,None,SREETS_NAMING_ADDRESSING_4815083.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.53736 19.51312, -99.53687 19.5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760184,None,974863297,None,None,None,None,None,None,None,None,SPA,CAMINO A LAS ÁNIMAS,LAS ÁNIMAS,None,None,N,CAMINO A,SREETS_NAMING_ADDRESSING_4815429.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.19933 19.72583, -99.19933 19.7..."
760185,None,1327241692,None,None,None,None,None,None,None,None,None,None,None,None,None,N,None,SREETS_NAMING_ADDRESSING_4815429.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.20038 19.72598, -99.20006 19.7..."
760186,None,1247247284,None,None,None,None,None,None,None,None,SPA,CALLE CONVENTO,CONVENTO,None,None,N,CALLE,SREETS_NAMING_ADDRESSING_4815429.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.20004 19.72786, -99.19999 19.7..."
760187,None,779540216,None,None,None,None,None,None,None,None,SPA,CALLE MANZANA 4 LOTE 13,MANZANA 4 LOTE 13,None,None,N,CALLE,SREETS_NAMING_ADDRESSING_4815429.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.20004 19.72786, -99.20039 19.7..."


In [15]:
snav_data = gpd.read_file('streets_nav.geojson')
snav_data

,AR_AUTO,AR_BUS,AR_CARPOOL,AR_DELIV,AR_EMERVEH,AR_MOTOR,AR_PEDEST,AR_TAXIS,AR_TRAFF,AR_TRUCKS,...,SPEED_CAT,TOLLWAY,TO_LANES,TO_SPD_LIM,TUNNEL,UNDEFTRAFF,URBAN,source_file,source_folder,geometry
0,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,7,N,0,0,N,N,Y,SREETS_NAV_4815440.geojson,STREETS_NAV,"LINESTRING (-99.13761 19.6875, -99.1376 19.68753)"
1,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,7,N,0,0,N,N,Y,SREETS_NAV_4815440.geojson,STREETS_NAV,"LINESTRING (-99.13886 19.6875, -99.13873 19.68..."
2,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,7,N,0,0,N,N,Y,SREETS_NAV_4815440.geojson,STREETS_NAV,"LINESTRING (-99.13886 19.6875, -99.14062 19.68..."
3,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,7,N,0,0,N,N,Y,SREETS_NAV_4815440.geojson,STREETS_NAV,"LINESTRING (-99.13873 19.68797, -99.13879 19.6..."
4,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,7,N,0,0,N,N,Y,SREETS_NAV_4815440.geojson,STREETS_NAV,"LINESTRING (-99.13879 19.68798, -99.13921 19.6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760184,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,7,N,0,0,N,N,Y,SREETS_NAV_4815098.geojson,STREETS_NAV,"LINESTRING (-98.9962 19.68723, -98.99601 19.6875)"
760185,Y,Y,Y,Y,Y,Y,Y,Y,N,Y,...,7,N,0,0,N,N,Y,SREETS_NAV_4815098.geojson,STREETS_NAV,"LINESTRING (-98.99281 19.6874, -98.99275 19.68..."
760186,Y,Y,Y,Y,Y,Y,Y,Y,N,Y,...,7,N,0,0,N,N,Y,SREETS_NAV_4815098.geojson,STREETS_NAV,"LINESTRING (-98.99281 19.6874, -98.99295 19.68..."
760187,Y,Y,Y,Y,Y,Y,Y,Y,N,Y,...,7,N,0,0,N,N,Y,SREETS_NAV_4815098.geojson,STREETS_NAV,"LINESTRING (-98.99295 19.68746, -98.99301 19.6..."


In [9]:
newdata = gpd.read_file('merged_streets_data.geojson')
newdata

,ADDR_TYPE,link_id,L_ADDRFORM,L_ADDRSCH,L_NREFADDR,L_REFADDR,R_ADDRFORM,R_ADDRSCH,R_NREFADDR,R_REFADDR,...,RAMP,ROUNDABOUT,SPEED_CAT,TOLLWAY,TO_LANES,TO_SPD_LIM,TUNNEL,UNDEFTRAFF,URBAN,geometry
0,None,1356100350,None,None,None,None,None,None,None,None,...,None,None,None,None,NaN,NaN,None,None,None,"LINESTRING (-99.53179 19.51172, -99.53184 19.5..."
1,None,1356100347,None,None,None,None,None,None,None,None,...,None,None,None,None,NaN,NaN,None,None,None,"LINESTRING (-99.53872 19.51172, -99.53862 19.5..."
2,None,1356100338,None,None,None,None,None,None,None,None,...,None,None,None,None,NaN,NaN,None,None,None,"LINESTRING (-99.54132 19.51172, -99.54134 19.5..."
3,None,1356100349,None,None,None,None,None,None,None,None,...,None,None,None,None,NaN,NaN,None,None,None,"LINESTRING (-99.53243 19.51222, -99.53252 19.5..."
4,None,1356100348,None,None,None,None,None,None,None,None,...,None,None,None,None,NaN,NaN,None,None,None,"LINESTRING (-99.53736 19.51312, -99.53687 19.5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1520373,None,780215040,None,None,None,None,None,None,None,None,...,N,N,7,N,0.0,0.0,N,N,Y,"LINESTRING (-98.9962 19.68723, -98.99601 19.6875)"
1520374,None,779725211,None,None,None,None,None,None,None,None,...,N,Y,7,N,0.0,0.0,N,N,Y,"LINESTRING (-98.99281 19.6874, -98.99275 19.68..."
1520375,None,1116108078,None,None,None,None,None,None,None,None,...,N,N,7,N,0.0,0.0,N,N,Y,"LINESTRING (-98.99281 19.6874, -98.99295 19.68..."
1520376,None,1116108079,None,None,None,None,None,None,None,None,...,N,N,7,N,0.0,0.0,N,N,Y,"LINESTRING (-98.99295 19.68746, -98.99301 19.6..."


In [20]:
pois = pd.read_csv('Student Sample Data/combined_pois.csv')
pois

/var/folders/m2/3gj76pxj4137lqnxbvjtp2h40000gn/T/ipykernel_60734/21748834.py:1: DtypeWarning: Columns (9,10,27,32) have mixed types. Specify dtype option on import or set low_memory=False.
  pois = pd.read_csv('Student Sample Data/combined_pois.csv')


,Unnamed: 0,LINK_ID,POI_ID,SEQ_NUM,FAC_TYPE,POI_NAME,POI_LANGCD,POI_NMTYPE,POI_ST_NUM,ST_NUM_FUL,...,ENTR_TYPE,REST_TYPE,FOOD_TYPE,ALT_FOOD,REG_FOOD,RSTR_TYPE,OPEN_24,DIESEL,BLD_TYPE,source_file
0,184337,1341716691,1222658906,1,8211,HERIBERTO ENRIQUEZ RODRÍGUEZ,SPA,B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POI_4815425.csv
1,175590,1130436938,1222845410,1,8211,ESCUELA GENERAL MARIANO ARISTA,SPA,B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POI_4815425.csv
2,188562,1197825736,1222679567,1,8211,ESCUELA NARCISO MENDOZA,SPA,B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POI_4815425.csv
3,188137,1197825732,1222779306,1,8211,OFTV NO 0253 ANTONIO ALZATE RAMÍREZ,SPA,B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POI_4815425.csv
4,188127,1123269751,1222745582,1,8211,ESCUELA EL AGUILA,SPA,B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POI_4815425.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193071,173012,779668663,1059481420,1,9992,PARROQUIA DE SAN MIGUEL ARCÁNGEL,SPA,B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHURCH,POI_4815429.csv
193072,180793,779668658,1165721462,1,8211,JAR DE NIN HANS CHRISTIAN ANDERSEN,SPA,B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POI_4815429.csv
193073,106460,779668658,1244376447,1,9565,FARMACIA,SPA,B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POI_4815429.csv
193074,164962,779666324,1209841744,1,9992,IGLESIA VIENTO RECIO MELCHOR OCAMPO,SPA,B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHURCH,POI_4815429.csv


In [22]:
# Find and display the first 6 duplicate rows in the POIs dataframe
duplicate_rows = pois[pois.duplicated(keep=False)].head(6)
print("\nFirst 6 duplicate rows:")
print(duplicate_rows)

# Count total number of duplicates
total_duplicates = pois.duplicated(keep=False).sum()
print(f"\nTotal number of duplicate rows: {total_duplicates}")

# Count duplicates by source file
if 'source_file' in pois.columns:
    duplicate_sources = pois[pois.duplicated(keep=False)]['source_file'].value_counts()
    print("\nDuplicates by source file:")
    print(duplicate_sources)
    # Find duplicates in POI_ID column
    poi_id_duplicates = pois[pois.duplicated(subset=['POI_ID'], keep=False)]
    
    print("\nFirst 6 duplicate POI_IDs:")
    print(poi_id_duplicates[['POI_ID', 'source_file']].head(6))
    
    # Count total number of duplicate POI_IDs
    total_poi_duplicates = poi_id_duplicates['POI_ID'].nunique()
    print(f"\nTotal number of unique POI_IDs that have duplicates: {total_poi_duplicates}")
    
    # Count duplicates by source file for POI_IDs
    poi_duplicate_sources = poi_id_duplicates['source_file'].value_counts()
    print("\nPOI_ID duplicates by source file:")
    print(poi_duplicate_sources)




First 6 duplicate rows:
Empty DataFrame
Columns: [Unnamed: 0, LINK_ID, POI_ID, SEQ_NUM, FAC_TYPE, POI_NAME, POI_LANGCD, POI_NMTYPE, POI_ST_NUM, ST_NUM_FUL, ST_NFUL_LC, ST_NAME, ST_LANGCD, POI_ST_SD, ACC_TYPE, PH_NUMBER, CHAIN_ID, NAT_IMPORT, PRIVATE, IN_VICIN, NUM_PARENT, NUM_CHILD, PERCFRREF, VANCITY_ID, ACT_ADDR, ACT_LANGCD, ACT_ST_NAM, ACT_ST_NUM, ACT_ADMIN, ACT_POSTAL, AIRPT_TYPE, ENTR_TYPE, REST_TYPE, FOOD_TYPE, ALT_FOOD, REG_FOOD, RSTR_TYPE, OPEN_24, DIESEL, BLD_TYPE, source_file]
Index: []

[0 rows x 41 columns]

Total number of duplicate rows: 0

Duplicates by source file:
Series([], Name: count, dtype: int64)

First 6 duplicate POI_IDs:
        POI_ID      source_file
10  1209839231  POI_4815425.csv
11  1209839231  POI_4815425.csv
14  1209839237  POI_4815425.csv
15  1209839237  POI_4815425.csv
19  1209839361  POI_4815425.csv
20  1209839361  POI_4815425.csv

Total number of unique POI_IDs that have duplicates: 5956

POI_ID duplicates by source file:
source_file
POI_4815085.csv

In [23]:
# Create a dataframe with only POI_ID duplicates and their corresponding observations
poi_duplicates = pois[pois.duplicated(subset=['POI_ID'], keep=False)].sort_values('POI_ID')

# Display the first few rows to verify
print("\nFirst few rows of POI_ID duplicates:")
print(poi_duplicates.head())

# Display basic information about the duplicates
print(f"\nTotal number of rows with duplicate POI_IDs: {len(poi_duplicates)}")
print(f"Number of unique POI_IDs that have duplicates: {poi_duplicates['POI_ID'].nunique()}")

# Display the dataframe
poi_duplicates



First few rows of POI_ID duplicates:
        Unnamed: 0    LINK_ID     POI_ID  SEQ_NUM  FAC_TYPE  \
7605          4285  703502036  800025481        2      7999   
7604          4284  703502036  800025481        1      7999   
164401      122118  703384522  800025539        2      5800   
164402      122117  703384522  800025539        1      5800   
66692       165587  703357464  800025585        1      7994   

                                     POI_NAME POI_LANGCD POI_NMTYPE  \
7605                                     科约阿坎        CHI          E   
7604                                 COYOACÁN        SPA          B   
164401           PIZZA HUT SUC 185 MONTEVIDEO        SPA          S   
164402                              PIZZA HUT        SPA          B   
66692   CENTRO COMUNITARIO DE SALUD MIRAVALLE        SPA          B   

        POI_ST_NUM ST_NUM_FUL  ... ENTR_TYPE REST_TYPE FOOD_TYPE ALT_FOOD  \
7605           NaN        NaN  ...       NaN       NaN       NaN      NaN   
76

,Unnamed: 0,LINK_ID,POI_ID,SEQ_NUM,FAC_TYPE,POI_NAME,POI_LANGCD,POI_NMTYPE,POI_ST_NUM,ST_NUM_FUL,...,ENTR_TYPE,REST_TYPE,FOOD_TYPE,ALT_FOOD,REG_FOOD,RSTR_TYPE,OPEN_24,DIESEL,BLD_TYPE,source_file
7605,4285,703502036,800025481,2,7999,科约阿坎,CHI,E,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POI_4815085.csv
7604,4284,703502036,800025481,1,7999,COYOACÁN,SPA,B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POI_4815085.csv
164401,122118,703384522,800025539,2,5800,PIZZA HUT SUC 185 MONTEVIDEO,SPA,S,NaN,NaN,...,NaN,NaN,PIZZA,NaN,NaN,TAKE-OUT & DELIVERY ONLY,NaN,NaN,NaN,POI_4815096.csv
164402,122117,703384522,800025539,1,5800,PIZZA HUT,SPA,B,NaN,NaN,...,NaN,NaN,PIZZA,NaN,NaN,TAKE-OUT & DELIVERY ONLY,NaN,NaN,NaN,POI_4815096.csv
66692,165587,703357464,800025585,1,7994,CENTRO COMUNITARIO DE SALUD MIRAVALLE,SPA,B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POI_4815090.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33728,234,1295217907,1262885797,2,4100,TERMINAL ESTACIÓN LOS PINOS,SPA,J,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POI_4815085.csv
89851,141038,1295380264,1262891174,2,7538,"P.A. AUTO HAUS, S.A. DE C.V. (ALAME",SPA,J,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POI_4815087.csv
89854,141037,1295380264,1262891174,1,7538,"P.A. AUTO HAUS, S.A. DE C.V. (ALAMEDAS)",SPA,B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POI_4815087.csv
89852,141009,1295380264,1262891175,2,5511,"P.A. AUTO HAUS, S.A. DE C.V. (ALAME",SPA,J,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POI_4815087.csv


In [24]:
# Drop duplicates based on POI_ID, keeping rows with longer POI_NAME and excluding CHI language
pois_cleaned = pois.copy()

# First, exclude rows where POI_LANGCD is 'CHI'
pois_cleaned = pois_cleaned[pois_cleaned['POI_LANGCD'] != 'CHI']

# Sort by POI_NAME length (descending) to keep longer names
pois_cleaned['name_length'] = pois_cleaned['POI_NAME'].str.len()
pois_cleaned = pois_cleaned.sort_values('name_length', ascending=False)

# Drop duplicates keeping the first occurrence (which will have the longest name)
pois_cleaned = pois_cleaned.drop_duplicates(subset=['POI_ID'], keep='first')

# Remove the temporary name_length column
pois_cleaned = pois_cleaned.drop('name_length', axis=1)

# Display results
print(f"Original number of rows: {len(pois)}")
print(f"Number of rows after cleaning: {len(pois_cleaned)}")
print(f"Number of duplicates removed: {len(pois) - len(pois_cleaned)}")

# Display the cleaned dataframe
pois_cleaned



Original number of rows: 193076
Number of rows after cleaning: 186669
Number of duplicates removed: 6407


,Unnamed: 0,LINK_ID,POI_ID,SEQ_NUM,FAC_TYPE,POI_NAME,POI_LANGCD,POI_NMTYPE,POI_ST_NUM,ST_NUM_FUL,...,ENTR_TYPE,REST_TYPE,FOOD_TYPE,ALT_FOOD,REG_FOOD,RSTR_TYPE,OPEN_24,DIESEL,BLD_TYPE,source_file
106740,172163,851035131,1210140360,1,9992,IGLESIA PUERTA DE SALVACIÓN ALABANZA Y ADORACI...,SPA,B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHURCH,POI_4815097.csv
179376,190228,702764303,1059254882,1,8211,ESCUELA SECUNDARIA TÉCNICA INDUSTRIAL Y COMERC...,SPA,B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POI_4815098.csv
191468,103083,702706827,1244216018,1,9567,TIENDA ESCOLAR ESCUELA SECUNDARIA OF 323 JOSE ...,SPA,B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POI_4815075.csv
130618,166387,1310846005,1206969826,1,9525,FISCALÍA DESCONCENTRADA DE INVESTIGACIÓN DE IZ...,SPA,B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POI_4815096.csv
142290,172344,703476040,1210138617,1,9992,TEMPLO EVANGÉLICO JESUS EN SAMARIA DE LAS ASAM...,SPA,B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHURCH,POI_4815096.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79289,168339,703604898,1200752158,1,9992,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHURCH,POI_4815079.csv
98721,191770,702800883,1226322495,1,7520,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POI_4815087.csv
138041,191841,949507455,1163109904,1,7520,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POI_4815096.csv
151049,191504,703439151,1224497309,1,7520,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POI_4815096.csv


In [27]:
# 2. Preparar los merges preservando geometrías
# Separar geometría y atributos de cada GeoDataFrame
snav_attrs = snav_data.drop(columns=['geometry'])
snav_geom = snav_data[['link_id', 'geometry']].rename(columns={'geometry': 'geom_snav'})

sn_attrs = sn_data.drop(columns=['geometry']) 
sn_geom = sn_data[['link_id', 'geometry']].rename(columns={'geometry': 'geom_sn'})

# 3. Hacer los merges secuenciales
# Primero agregar atributos de snav_data
pois_with_snav = pois_cleaned.merge(
    snav_attrs, 
    left_on='LINK_ID', 
    right_on='link_id', 
    how='left',
    suffixes=('', '_snav')
)

# Luego agregar atributos de sn_data
pois_with_attrs = pois_with_snav.merge(
    sn_attrs,
    left_on='LINK_ID', 
    right_on='link_id', 
    how='left',
    suffixes=('', '_sn')
)

# 4. Agregar geometrías
pois_with_snav_geom = pois_with_attrs.merge(
    snav_geom,
    left_on='LINK_ID', 
    right_on='link_id', 
    how='left'
)

pois_final = pois_with_snav_geom.merge(
    sn_geom,
    left_on='LINK_ID', 
    right_on='link_id', 
    how='left'
)

# 5. Limpiar columnas redundantes
columns_to_drop = [col for col in pois_final.columns if col.startswith('link_id') and col != 'link_id']
pois_final = pois_final.drop(columns=columns_to_drop, errors='ignore')

# 6. Convertir a GeoDataFrame si quieres trabajar con las geometrías
# Usar una de las geometrías como principal (por ejemplo, la de snav_data)
pois_geo_final = gpd.GeoDataFrame(pois_final, geometry='geom_snav')

In [28]:
pois_geo_final

,Unnamed: 0,LINK_ID,POI_ID,SEQ_NUM,FAC_TYPE,POI_NAME,POI_LANGCD,POI_NMTYPE,POI_ST_NUM,ST_NUM_FUL,...,ST_NM_BASE,ST_NM_SUFF,ST_TYP_AFT,ST_TYP_ATT,ST_TYP_BEF,source_file_sn,source_folder_sn,geom_snav,link_id,geom_sn
0,172163,851035131,1210140360,1,9992,IGLESIA PUERTA DE SALVACIÓN ALABANZA Y ADORACI...,SPA,B,NaN,NaN,...,None,None,None,N,None,SREETS_NAMING_ADDRESSING_4815097.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-98.90419 19.38114, -98.90412 19.3...",851035131,"LINESTRING (-98.90419 19.38114, -98.90412 19.3..."
1,190228,702764303,1059254882,1,8211,ESCUELA SECUNDARIA TÉCNICA INDUSTRIAL Y COMERC...,SPA,B,NaN,NaN,...,MARTÍN GARCÍA,None,None,N,CALLE,SREETS_NAMING_ADDRESSING_4815098.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.0122 19.60828, -99.01298 19.60...",702764303,"LINESTRING (-99.0122 19.60828, -99.01298 19.60..."
2,103083,702706827,1244216018,1,9567,TIENDA ESCOLAR ESCUELA SECUNDARIA OF 323 JOSE ...,SPA,B,NaN,NaN,...,INDEPENDENCIA,None,None,N,AVENIDA,SREETS_NAMING_ADDRESSING_4815075.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.62585 19.3116, -99.6259 19.31291)",702706827,"LINESTRING (-99.62585 19.3116, -99.6259 19.31291)"
3,166387,1310846005,1206969826,1,9525,FISCALÍA DESCONCENTRADA DE INVESTIGACIÓN DE IZ...,SPA,B,NaN,NaN,...,TELECOMUNICACIONES,None,None,N,PROLONGACIÓN,SREETS_NAMING_ADDRESSING_4815096.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.04152 19.38159, -99.04188 19.3...",1310846005,"LINESTRING (-99.04152 19.38159, -99.04188 19.3..."
4,172344,703476040,1210138617,1,9992,TEMPLO EVANGÉLICO JESUS EN SAMARIA DE LAS ASAM...,SPA,B,NaN,NaN,...,NORTE 88,None,None,N,CALLE,SREETS_NAMING_ADDRESSING_4815096.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.10348 19.45269, -99.10304 19.4...",703476040,"LINESTRING (-99.10348 19.45269, -99.10304 19.4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202054,168339,703604898,1200752158,1,9992,NaN,NaN,NaN,NaN,NaN,...,LACANDONES,None,None,N,CALLE,SREETS_NAMING_ADDRESSING_4815079.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.17665 19.27189, -99.1773 19.27...",703604898,"LINESTRING (-99.17665 19.27189, -99.1773 19.27..."
202055,191770,702800883,1226322495,1,7520,NaN,NaN,NaN,NaN,NaN,...,MARIANO ESCOBEDO,None,None,N,CALLE,SREETS_NAMING_ADDRESSING_4815087.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.19853 19.53977, -99.19971 19.5...",702800883,"LINESTRING (-99.19853 19.53977, -99.19971 19.5..."
202056,191841,949507455,1163109904,1,7520,NaN,NaN,NaN,NaN,NaN,...,FRAY SERVANDO TERESA DE MIER,None,None,N,AVENIDA,SREETS_NAMING_ADDRESSING_4815096.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.09945 19.41668, -99.09974 19.4...",949507455,"LINESTRING (-99.09945 19.41668, -99.09974 19.4..."
202057,191504,703439151,1224497309,1,7520,NaN,NaN,NaN,NaN,NaN,...,BRAVO,None,None,N,CALLE,SREETS_NAMING_ADDRESSING_4815096.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.1217 19.44017, -99.12162 19.4408)",703439151,"LINESTRING (-99.1217 19.44017, -99.12162 19.4408)"


In [29]:
pois_geo_final.to_csv('dataset_final.csv', index=False)

In [31]:
import pandas as pd

data = pd.read_csv('dataset_final.csv')
data

/var/folders/m2/3gj76pxj4137lqnxbvjtp2h40000gn/T/ipykernel_60734/349556683.py:3: DtypeWarning: Columns (9,10,27,32,35,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('dataset_final.csv')


,Unnamed: 0,LINK_ID,POI_ID,SEQ_NUM,FAC_TYPE,POI_NAME,POI_LANGCD,POI_NMTYPE,POI_ST_NUM,ST_NUM_FUL,...,ST_NM_BASE,ST_NM_SUFF,ST_TYP_AFT,ST_TYP_ATT,ST_TYP_BEF,source_file_sn,source_folder_sn,geom_snav,link_id,geom_sn
0,172163,851035131,1210140360,1,9992,IGLESIA PUERTA DE SALVACIÓN ALABANZA Y ADORACI...,SPA,B,NaN,NaN,...,NaN,NaN,NaN,N,NaN,SREETS_NAMING_ADDRESSING_4815097.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-98.90419 19.38114, -98.90412 19.3...",851035131,"LINESTRING (-98.90419 19.38114, -98.90412 19.3..."
1,190228,702764303,1059254882,1,8211,ESCUELA SECUNDARIA TÉCNICA INDUSTRIAL Y COMERC...,SPA,B,NaN,NaN,...,MARTÍN GARCÍA,NaN,NaN,N,CALLE,SREETS_NAMING_ADDRESSING_4815098.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.0122 19.60828, -99.01298 19.60...",702764303,"LINESTRING (-99.0122 19.60828, -99.01298 19.60..."
2,103083,702706827,1244216018,1,9567,TIENDA ESCOLAR ESCUELA SECUNDARIA OF 323 JOSE ...,SPA,B,NaN,NaN,...,INDEPENDENCIA,NaN,NaN,N,AVENIDA,SREETS_NAMING_ADDRESSING_4815075.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.62585 19.3116, -99.6259 19.31291)",702706827,"LINESTRING (-99.62585 19.3116, -99.6259 19.31291)"
3,166387,1310846005,1206969826,1,9525,FISCALÍA DESCONCENTRADA DE INVESTIGACIÓN DE IZ...,SPA,B,NaN,NaN,...,TELECOMUNICACIONES,NaN,NaN,N,PROLONGACIÓN,SREETS_NAMING_ADDRESSING_4815096.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.04152 19.38159, -99.04188 19.3...",1310846005,"LINESTRING (-99.04152 19.38159, -99.04188 19.3..."
4,172344,703476040,1210138617,1,9992,TEMPLO EVANGÉLICO JESUS EN SAMARIA DE LAS ASAM...,SPA,B,NaN,NaN,...,NORTE 88,NaN,NaN,N,CALLE,SREETS_NAMING_ADDRESSING_4815096.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.10348 19.45269, -99.10304 19.4...",703476040,"LINESTRING (-99.10348 19.45269, -99.10304 19.4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202054,168339,703604898,1200752158,1,9992,NaN,NaN,NaN,NaN,NaN,...,LACANDONES,NaN,NaN,N,CALLE,SREETS_NAMING_ADDRESSING_4815079.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.17665 19.27189, -99.1773 19.27...",703604898,"LINESTRING (-99.17665 19.27189, -99.1773 19.27..."
202055,191770,702800883,1226322495,1,7520,NaN,NaN,NaN,NaN,NaN,...,MARIANO ESCOBEDO,NaN,NaN,N,CALLE,SREETS_NAMING_ADDRESSING_4815087.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.19853 19.53977, -99.19971 19.5...",702800883,"LINESTRING (-99.19853 19.53977, -99.19971 19.5..."
202056,191841,949507455,1163109904,1,7520,NaN,NaN,NaN,NaN,NaN,...,FRAY SERVANDO TERESA DE MIER,NaN,NaN,N,AVENIDA,SREETS_NAMING_ADDRESSING_4815096.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.09945 19.41668, -99.09974 19.4...",949507455,"LINESTRING (-99.09945 19.41668, -99.09974 19.4..."
202057,191504,703439151,1224497309,1,7520,NaN,NaN,NaN,NaN,NaN,...,BRAVO,NaN,NaN,N,CALLE,SREETS_NAMING_ADDRESSING_4815096.geojson,STREETS_NAMING_ADDRESSING,"LINESTRING (-99.1217 19.44017, -99.12162 19.4408)",703439151,"LINESTRING (-99.1217 19.44017, -99.12162 19.4408)"
